# Quiz Generator

Generator **Moodle** *Embedded Answer (Clotz)* kviza.

Celoten opis izgradnje takega kviza je na <a href="https://docs.moodle.org/en/Embedded_Answers_(Cloze)_question_type">Moodle Embedded Answer (Clotz)</a>.

V našem primeru gradimo samo numeričen tip (**NUMERICAL**), vendar ima možnost vrsto različnih tipov. Za igradnjo enostavnih kvizov se uporablja tudi vmesnik [Hot Potatoes](https://docs.moodle.org/311/en/Hot_Potatoes).

In [1]:
# Nalaganje potrebnih paketoc
import sys

# sources path
sys.path.append('../src')

import random as rnd
import quizgenerator as qg

## Utils functions

Pomožne funkcije za gradnj naključnih števil

In [2]:
# Generate random number
# y = x + rand(dx)
# rand(dx) is in the interval [-dx,dx]
def randValue(x,dx,N):
    
    return x + (2*rnd.randint(0,N) - N)/N * dx

In [3]:
# Returns question random values for inputs and for results
def getQuestionRandomValues(p_inp, nq):
    
    res = []
    for i in range(nq):
        y = modelFunction(p_inp)
        res.append(y)
        
    return res

## Input functions

Funkcije, s katerimi zgradimo potrebne vhodne parametre.

- *modelFunction()*: tukaj poteka izračun rezultatov na podlagi vhodnih parametrov. Iz vhodnih parametrov se zgenerirajo vhodna naključna števil, ki jih nato uporabimo za izračun rezultatov,

- *getInputParameters()*: postavitev zgradbo vhodnih parametrov, ki so določeni s predpisanim formatom **[$x$, $\Delta x$, u, name, format]**
    + $x$: srednja vrednost parametra,
    + $\Delta x$: absolutni oplet, kjer je to naključno število v intervalu $[-\Delta x, \Delta x]$ ($y = x \pm \mathrm{rnd}(-1,1) \cdot \Delta x$),
    + u: enota parametra,
    + name: ime parametra,
    + format: format načina izpisa
<br>
<br>

- *getOutputParameters()*: postavitev zgradbe rezultatov, ki so določeni s predpisanim formatom **[p, e, u, format, name, comment]**
    + p: vrednost odgovora v točkah ali utež (celo število),
    + e: **absolutna napaka**, ki jo dopuščamo pri rezultatu študenta,
    + u: enota rezultata,
    + format: format načina izpisa,
    + name: ime parametra,
    + comment: možen komentar ob rezultatu

In [4]:
# Model function based on input definitions
# - generate random inputs
# - generate results based on random inputs
def modelFunction(p_inp):
    N = 10000
    x = []
    
    # generate random numbers for inputs x
    for p in p_inp:
        v = randValue(p[0],p[1],N)
        x.append(v)
    
    # *** random inputs ***
    D0 = x[0]  # inital displacement
    Tf0 = x[1] # initial draft forward
    Ta0 = x[2] # initial draft afterward
    TPC = x[3] # TPC 
    m = x[4]   # discharged cargo mass (negative sign)
    rand_inp = [D0,Tf0,Ta0,TPC,m]
    
    # *** results ***
    Ts0 = (Tf0 + Ta0)/2 # initial mean draft
    dTs =  -m / TPC     # draft change
    D1 = D0 - m         # new displacement
    Ts1 = Ts0 + dTs/100 # new mean draft
    rand_res = [Ts0,dTs,D1,Ts1]
    
    return [rand_inp, rand_res]

In [5]:
# generates input parameters
def getInputParameters():
    # *** define inputs ***
    #
    # [x, dx, u, name, format]:
    #  x: mean,
    #  dx: random interval
    #  u: unit
    #  name: input variable name (possible use LaTeX form, i.e. \Delta)
    #  format: specify output format, based on formatted strings
    #
    # random value is calculated with function randValue(x,dx)
    
    D = [60000, 5000, 't',r'\Delta','.1f']          # displacement
    Tf = [10.5, 0.5, 'm',r'T_\mathrm{f}','.2f']     # draft forward
    Ta = [9.5, 0.5, 'm',r'T_\mathrm{a}','.2f']      # draft aft       
    TPC = [20.0, 2.0, 't/cm',r'\mathrm{TPC}','.2f'] # tone per centimeter - TPC
    m = [1000.0, 200.0, 't',r'\mathrm{m}','.2f']    # mass
    
    return [D, Tf, Ta, TPC, m]

In [6]:
# generate output parameters
def getOutputParameters():
    # *** define outputs properties ***
    #
    # [p, e, u, format, name, comment]:
    #  p: number of points for correct answer (must be integer)
    #  e: abslute error
    #  u: unit
    #  format: specify output format, based on formatted strings
    #  name: output variable name (possible use LaTeX form, i.e. \Delta)
    #  comment: comments related to result (description, points, ...)
    #
    # This are results of the Quiz!
    
    Ts0 = [1,0.02,'m','.2f',r'T_\mathrm{s}','začetni srednji ugrez; 1 točka']
    dTs = [2,2,'cm','.2f', r'\Delta T','sprememba ugreza; 2 točki']
    D1 = [1,5,'t','.1f',r'\Delta','deplasman na koncu; 1 točka']     
    Ts1 = [1,0.02,'m','.2f',r'T_\mathrm{s}','srednji ugrez na koncu; 2 točki']
    
    return [Ts0,dTs,D1,Ts1]

## Steering program part

Nastavitve v katero kategorijo se naj vprašanja zapišejo. Pod-kategorija bo tako vsebovala ista vprašanja, vendar z različnimi vhodnimi parametri. Tako za isto nalogo dobimo različne rezultate, kar onemogoča prepisovanje, razen če študent ne preplonka celotnega postopka reševanja, torej že nekaj zna!

- *category_path*: kategorija kam vprašanje spada, kjer kategorije gnezdimo z znakom *slash*-"/",
- *question_ID*: to je ključ po katerem shranim vprašanja v svojo pad-kategorijo
- *question_form_path*: pot do datoteke, ki vsebuje formo vprašanja

**Forma vprašanja** je sestavljena iz teksta, kjer mesto vhodnih parametrov označimo z **{inp:#}** in mesto rezultatov **{out:#}**. Lojtra je tuaj zaporedna številka, s katero vemo kateri parameter je to. zaporedna številka je vezana na mesto, ki ga ima parameter v defininiciji, kot je narejeno v **Input functions**!

Celoten kviz, z vsemi naključnimi vprašanji se nato shrani v datoteko, ki ji določite ime pod spremenljivko **quiz_file_name**!

**Število vprašanj** se nastavi s spodnjim parametrom **Nq**!

In [7]:
# input parameters
p_inp = getInputParameters()

# output parameters
p_out = getOutputParameters()

# questions random values ([inputs, results]) 
Nq = 10 # number of question random realizations
p_rand = getQuestionRandomValues(p_inp, Nq) # random results


# *** quiz parameters ***
category_path = 'test/2022'
question_ID = 'q-01'
question_form_path = '../examples/quiz_form_01.txt'

# *** initial Quiz ***
quiz = qg.QuizGenerator(category_path, question_ID, question_form_path)
# generate randomized Quiz
quiz.generateQuiz(p_inp, p_out, p_rand)

Question form example:
  -> file abs path: /Users/sandro/Cloud/github_as-grm/moodle_quiz_generator/examples/quiz_form_01.txt
  -> file base path: /Users/sandro/Cloud/github_as-grm/moodle_quiz_generator/examples
  -> file name: quiz_form_01.txt
Question form summary:
  ->  inputs found: ['{inp:1}', '{inp:2}', '{inp:3}', '{inp:4}', '{inp:5}']
  -> outputs found: ['{out:1}', '{out:2}', '{out:3}', '{out:4}']

Generator summary:
  -> generated 10 different questions
  -> Quiz is written in "/Users/sandro/Cloud/github_as-grm/moodle_quiz_generator/examples/quiz_form_01.xml" XML file
  -> Quiz is now ready for import in Moodle. Import needs XML format importer!
